# 모델 불러오기

In [ ]:
import dill

import os
import random

import pandas as pd
import numpy as np

from sklearn.metrics import roc_curve, auc

import torch
import torch.nn as nn

import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize

from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import BucketIterator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Nondeterministic 한 작업 피하기
RANDOM_SEED = 2022

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True   # Deterministic 한 알고리즘만 사용하기
torch.backends.cudnn.benchmark = False      # Cudnn benchmark 해제
torch.cuda.manual_seed_all(RANDOM_SEED)     # if use multi-GPU

os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

# DATA_PATH = "data/processed/"
DATA_PATH = "/content/drive/Othercomputers/내 컴퓨터/Sat_english"

Pre_training 모델의 Class

In [ ]:
class LSTM_Model(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super().__init__()

        # Embedding Layer
        self.embed_layer = nn.Embedding(
            num_embeddings=num_embeddings, 
            embedding_dim=embedding_dim, 
            padding_idx=pad_idx
        )

        # LSTM Layer
        self.lstm_layer = nn.LSTM(
            input_size=embedding_dim, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first = True,
            bidirectional=True,     # 양방향 LSTM
            dropout=0.5
        )

        # Fully-connetcted Layer
        self.fc_layer1 = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),    # 양방향 LSTM의 출력은 입력의 2배
            nn.Dropout(0.5),
            nn.LeakyReLU()      # f(x)=max(0.01x, x)로 dying ReLU 방지
        )
        self.fc_layer2 = nn.Sequential(
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        embed_x = self.embed_layer(x)

        output, (_, _) = self.lstm_layer(embed_x)       # hidden, cell state의 출력값 사용 안함
        
        output = output[:, -1, :]       # (batch_size, seq_length, 2*hidden_size) -> (batch_size, 2*hidden_size)

        output = self.fc_layer1(output)
        output = self.fc_layer2(output)
        return output

Advanced_model의 Class

In [ ]:
class LSTM_Pooling_Model(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super().__init__()

        # Embedding Layer
        self.embed_layer = nn.Embedding(
                num_embeddings=num_embeddings,
                embedding_dim=embedding_dim,
                padding_idx=pad_idx
            )
        
        # LSTM Layer
        self.lstm_layer = nn.LSTM(
            input_size=embedding_dim, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            bidirectional=True,     # 양방향 LSTM
            batch_first=True,       
            dropout=0.5     
        )

        # Fully-connetcted Layer
        self.fc_layer = nn.Sequential(
            nn.Linear(hidden_size*2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.embed_layer(x)

        output, _ = self.lstm_layer(x)

        pool = nn.functional.max_pool1d(output.transpose(1, 2), x.shape[1])
        pool = pool.transpose(1, 2).squeeze()
        pool = nn.functional.dropout(pool, p=0.5)

        output = self.fc_layer(pool)

        return output.squeeze()

성능 비교 함수

In [ ]:
def test(model_path, device):
    # 저장한 모델 불러오기
    with open(model_path, "rb") as f:
        model = dill.load(f)
        
    # 데이터 불러오기
    sat_test_data = TabularDataset(
        path=f"{DATA_PATH}/data/processed/sat_test.tsv",
        format="tsv",
        fields=[("text", model["TEXT"]), ("label", model["LABEL"])],
        skip_header=1
    )

    # DataLoader 정의
    sat_test_iterator = BucketIterator(
        sat_test_data,
        batch_size=8, 
        device=None,
        sort=False,
        shuffle=False
    )

    # 모델에 입력
    classifier = model["classifier"]
    classifier.eval()
    with torch.no_grad():
        y_real = []
        y_pred = []

        for batch in sat_test_iterator:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)

            output = classifier(text).flatten().cpu()
 
            y_real += [label]
            y_pred += [output]

        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)

    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)

    return auroc.round(5)

성능 비교

In [ ]:
# 실제 저장한 모델 불러오기 및 테스트 진행
model_list = [
    f"{DATA_PATH}/before_tuning_model.dill",
    f"{DATA_PATH}/after_tuning_model.dill",
    f"{DATA_PATH}/advanced_before_tuning_model.dill",
    f"{DATA_PATH}/advanced_after_tuning_model.dill",
]

test_auroc = []
for file_name in model_list:
    model_name = file_name.replace(".dill", "")
    auroc = test(file_name, "cpu")
    test_auroc += [(model_name, auroc)]

In [ ]:
# 모델의 성능 순으로 정렬하여 표시
test_auroc = sorted(test_auroc, key=lambda x: x[1], reverse=True)
for rank, (model_name, auroc) in enumerate(test_auroc):
    print(f"Rank {rank+1} - {model_name:30} - Test AUROC: {auroc:.5f}")

Rank 1 - /content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model - Test AUROC: 0.88462
Rank 2 - /content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model - Test AUROC: 0.80769
Rank 3 - /content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model - Test AUROC: 0.73077
Rank 4 - /content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model - Test AUROC: 0.69231


# 실제 문제 풀어보기

문제 풀이 함수

In [ ]:
def predict_problem(model_path, problem):
    with open(model_path, "rb") as f:
        model = dill.load(f)
    TEXT = model["TEXT"]
    classifier = model["classifier"]

    problem = list(map(lambda x: x.replace("[", "").replace("]", ""), problem))     # 필요없는 기호 지우기
    
    # 문장 Token화
    tokenized_sentences = [word_tokenize(sentence) for sentence in problem]
    sentences = []
    for tokenized_sentence in tokenized_sentences:
        sentences.append([TEXT.vocab.stoi[word] for word in tokenized_sentence])    # TEXT에 들어있는 단어장 이용
    
    # 모델에 입력
    classifier.eval()
    with torch.no_grad():
        predict = []
        for sentence in sentences:
            sentence = torch.LongTensor([sentence])     # 64-bit int형 텐서로 선언
            predict += [classifier(sentence).item()]
    return predict

여러 모델을 처리하는 함수

In [ ]:
def predict_problem_with_models(model_list, problem):
    scores = {}

    # 각 모델별 결과 출력
    for file_name in model_list:
        model_name = file_name.replace(".dill", "")
        score = predict_problem(file_name, problem)
        scores[model_name] = score

    score_df = pd.DataFrame(scores).T
    score_df.columns = [f"answer_{i}_score" for i in range(1,6)]

    selected_answer = pd.Series(
            np.argmin(score_df.values, 1)+1,       # 각 모델별 가장 작은 Score값의 위치를 찾기 [(0~4)+1]
            index=score_df.index,
            name="selected_answer"
        )

    return pd.concat([selected_answer, score_df], 1)

고3 수능 및 평가원 모의고사 테스트

In [ ]:
problem_1 = [ 
    "Competitive activities can be more than just performance showcases which the best is recognized and the rest are overlooked.",
    "The provision of timely, constructive feedback to participants on performance is an asset that some competitions and contests offer.",
    "The provision of that type of feedback can be interpreted as shifting the emphasis to demonstrating superior performance but not necessarily excellence.",
    "The emphasis on superiority is what we typically see as fostering a detrimental effect of competition.",
    "Information about performance can be very helpful, not only to the participant who does not win or place but also to those who do.",
]
problem_1_label = [0, 1, 1, 1, 1]

In [ ]:
# 저장된 모델과 문제 입력 후 앞서 비교한 성능이 좋은 순서로 나열
predict_problem_with_models(model_list, problem_1).loc[map(lambda x:x[0], test_auroc)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]


,selected_answer,answer_1_score,answer_2_score,answer_3_score,answer_4_score,answer_5_score
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model,1,0.945917,0.998934,0.997799,0.998186,0.954952
/content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model,4,0.475727,0.475711,0.475811,0.475520,0.475803
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model,5,0.946149,0.999861,0.999371,0.998895,0.170715
/content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model,4,0.840244,0.827726,0.840919,0.809097,0.809818


In [ ]:
problem_2 = [ 
    "People from more individualistic cultural contexts tend to be motivated to maintain self-focused agency or control as these serve as the basis of one's self-worth.",
    "With this form of agency comes the belief that individual successes depending primarily on one's own abilities and actions, and thus, whether by influencing the environment or trying to accept one's circumstances, the use of control ultimately centers on the individual.",
    "The independent self may be more driven to cope by appealing to a sense of agency or control.",
    "Research has shown that East Asians prefer to receive, but not seek, more social support rather than seek personal neglect in certain cases.",
    "Therefore, people who hold a more interdependent self-construal may prefer to cope in a way that promotes harmony in relationships.",
]
problem_2_label = [1, 0, 1, 1, 1]

In [ ]:
predict_problem_with_models(model_list, problem_2).loc[map(lambda x:x[0], test_auroc)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]


,selected_answer,answer_1_score,answer_2_score,answer_3_score,answer_4_score,answer_5_score
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model,2,0.998325,0.959663,0.993074,0.984891,0.997952
/content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model,2,0.475745,0.475685,0.475784,0.475821,0.475787
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model,2,0.999908,0.974186,0.998819,0.986178,0.990818
/content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model,1,0.840637,0.840714,0.840801,0.840657,0.840730


In [ ]:
problem_3 = [ 
    "A cell is born as a twin when its mother cell divides, producing two daughter cells.",
    "Each daughter cell is smaller than the mother cell, and except for unusual cases, each grows until it becomes as large as the mother cell was",
    "After the cell has grown to the proper size, its metabolism shifts as it either prepares to divide or matures and differentiates into a specialized cell.",
    "What cell metabolism and structure should be complex would not be surprising, but actually, they are rather simple and logical.",
    "Even the most complex cell has only a small number of parts, each responsible for a distinct, well-defined aspect of cell life.",
]
problem_3_label = [1, 1, 1, 0, 1]

In [ ]:
predict_problem_with_models(model_list, problem_3).loc[map(lambda x:x[0], test_auroc)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]


,selected_answer,answer_1_score,answer_2_score,answer_3_score,answer_4_score,answer_5_score
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model,3,0.994853,0.998228,0.822611,0.921394,0.986120
/content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model,5,0.475759,2.359796,0.475768,0.475675,0.475657
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model,3,0.866024,0.998683,0.234920,0.999705,0.535214
/content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model,5,0.840955,0.872352,0.834410,0.834431,0.820522


In [ ]:
problem_4 = [ 
    "As far as communication betweewn humans is concerned, such commonality of interests is rarely achieved; even a pregnant mother has reasons to mistrust the chemical signals sent by her fetus.",
    "A prey can convince a predator not to chase it.",
    "But for such communication to occur, there must be strong guarantees which those who receive the signal will be better off believing it.",
    "The messages have to be kept, on the whole, honest.",
    "In the case of humans, honesty is maintained by a set of cognitive mechanisms that evaluate communicated information.",
]
problem_4_label = [1, 1, 0, 1, 1]

In [ ]:
predict_problem_with_models(model_list, problem_4).loc[map(lambda x:x[0], test_auroc)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]


,selected_answer,answer_1_score,answer_2_score,answer_3_score,answer_4_score,answer_5_score
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model,3,0.999503,0.998557,0.997283,0.998644,0.997673
/content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model,4,0.475775,0.475376,0.475586,0.475082,0.475735
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model,2,0.997901,0.996408,0.999879,0.999611,0.998789
/content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model,4,0.840748,0.809714,0.817176,0.805634,0.829604


In [ ]:
problem_5 = [
    "Early astronomy provided information about when to plant crops and gave humans their first formal method of recording the passage of time.",
    "Stonehenge, the 4000-year-old ring of stones in southern Britain, is perhaps the best-known monument to the discovery of regularity and predictability in the world we inhabit.",
    "The great markers of Stonehenge point to the spots on the horizon where the sun rises at the solstices and equinoxes - the dates we still use to mark the beginnings of the seasons.",
    "The stones may even have been used to predict eclipses.",
    "The existene of Stonehenge, built by people without writing, bears silent testimony both to the regularity of nature and to the ability of the human mind to see behind immediate appearances and discovers deeper meanings in events.",
]
problem_5_label = [1, 1, 1, 1, 0]

In [ ]:
predict_problem_with_models(model_list, problem_5).loc[map(lambda x:x[0], test_auroc)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]


,selected_answer,answer_1_score,answer_2_score,answer_3_score,answer_4_score,answer_5_score
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model,2,0.996779,0.635927,0.967725,0.993224,0.965123
/content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model,1,0.475611,0.475706,0.475704,0.475732,0.475819
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model,4,0.998051,0.942529,0.995258,0.638522,0.999749
/content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model,1,0.819598,0.840438,0.840729,0.840589,0.840930


고1 모의고사 테스트

In [ ]:
problem_6 = [
    "The money from anything that's produced is used to buy something else.",
    "There can never be a situation which a firm finds that it can't sell its goods and so has to dismiss workers and close its factories.",
    "Say's Law applies because people use all their earnings to buy things.",
    "But what happens if people don't spend all their money, saving some of it instead?",
    "That would mean firms producing less and dismissing some of their workers.",
]
problem_6_label = [1, 0, 1, 1, 1]

In [ ]:
predict_problem_with_models(model_list, problem_6).loc[map(lambda x:x[0], test_auroc)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]


,selected_answer,answer_1_score,answer_2_score,answer_3_score,answer_4_score,answer_5_score
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model,1,0.507870,0.997745,0.998938,0.914238,0.999103
/content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model,1,0.475313,0.475731,0.475839,0.484667,0.475390
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model,1,0.128006,0.995576,0.999052,0.301641,0.998380
/content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model,1,0.808279,0.840693,0.818843,0.833696,0.808408


In [ ]:
problem_7 = [
    "There have been occasions in which you have observed a smile and you could sense it was not genuine.",
    "The most obvious way of identifying a genuine smile from an insincere one is that a fake smile primarily only affects the lower half of the face, mainly with the mouth alone.",
    "Take the opportunity to look in the mirror and manufacture a smile using the lower half your face only.",
    "When you do this, judge how happy your face really looks ― is it genuine?",
    "A genuine smile will impact on the muscles and wrinkles around the eyes and less noticeably, the skin between the eyebrow and upper eyelid are lowered slightly with true enjoyment.",
]
problem_7_label = [1, 1, 1, 1, 0]

In [ ]:
predict_problem_with_models(model_list, problem_7).loc[map(lambda x:x[0], test_auroc)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]


,selected_answer,answer_1_score,answer_2_score,answer_3_score,answer_4_score,answer_5_score
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model,5,0.998886,0.999658,0.998735,0.997345,0.975063
/content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model,2,0.475360,0.475252,0.475629,0.482083,0.475806
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model,1,0.988410,0.989868,0.998554,0.997961,0.995203
/content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model,4,0.810353,0.811885,0.813025,0.793112,0.840913


In [ ]:
problem_8 = [
    "Although there is usually a correct way of holding and playing musical instruments, the most important instruction to begin with is that they are not toys and that they must be looked after.",
    "Allow children time to explore ways of handling and playing the instruments for themselves before showing them.",
    "Finding different ways to produce sounds are an important stage of musical exploration.",
    "Correct playing comes from the desire to find the most appropriate sound quality and find the most comfortable playing position so that one can play with control over time.",
    "As instruments and music become more complex, learning appropriate playing techniques becomes increasingly relevant.",
]
problem_8_label = [1, 1, 0, 1, 1]

In [ ]:
predict_problem_with_models(model_list, problem_8).loc[map(lambda x:x[0], test_auroc)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]


,selected_answer,answer_1_score,answer_2_score,answer_3_score,answer_4_score,answer_5_score
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_after_tuning_model,5,0.997621,0.998208,0.999295,0.997778,0.977652
/content/drive/Othercomputers/내 컴퓨터/Sat_english/after_tuning_model,1,0.475175,0.475800,0.475818,0.475602,0.475800
/content/drive/Othercomputers/내 컴퓨터/Sat_english/advanced_before_tuning_model,1,0.949038,0.997004,0.994877,0.997784,0.963441
/content/drive/Othercomputers/내 컴퓨터/Sat_english/before_tuning_model,1,0.806855,0.832348,0.840685,0.812817,0.840901
